In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 LangChain 總結大型文件 🦜🔗

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 圖示"><br>在 Colab 上執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 圖示"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 圖示"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Polong Lin](https://github.com/polong-lin) |


## 概述

文本摘要化是一項 NLP 任務，用以建立較長篇幅文本的簡明且富含資訊的摘要。LLM 可用於建立新聞文章、研究論文、技術文件和其他類型文本的摘要。

摘要化大型文件可能極具挑戰性。若要建立摘要，你需要針對索引文件套用摘要化策略。你已在之前的記事本中看過這些策略中的一些。如果你尚未完成，建議完成該作業以基本了解如何摘要化大型文件。

在本記事本中，你將使用 LangChain (開發 LLM 應用程式的架構) 套用一些摘要化策略。本記事本涵蓋多種如何摘要化大型文件的範例。


### 目標

在這個教學課程中，你將學習如何使用 LangChain 搭配 PaLM API 透過執行以下範例來摘要大型文件：

- 填充法
- MapReduce 方法
- 改善法


### 成本

此教學使用 Google Cloud 的計費元件：
- Vertex AI Generative AI Studio

進一步了解 [Vertex AI 價格](https://cloud.google.com/vertex-ai/pricing)，並使用 [價格計算器](https://cloud.google.com/products/calculator/) 根據你的預計使用量產生預估成本。


## 開始使用


### 安裝 Vertex AI SDK 和其他依賴項


In [ ]:
!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""

! pip3 install {USER} --upgrade pytesseract pypdf PyPDF2 textract langchain transformers google-cloud-aiplatform --quiet

**Colab 專屬** : 執行以下Cell，重新啟動核心。對於 Vertex AI Workbench，你可以使用最上方的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證你的筆記本環境

- 如果你使用 **Colab** 來執行這個筆記本，請執行以下單元格並繼續。
- 如果你使用的是 **Vertex AI Workbench** ，在此處查看設定說明 [link](https://github.com/doggy8088/generative-ai/tree/main/setup-env).


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 匯入函式庫


**僅限 Colab：** 執行下列Cell以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此操作。


In [ ]:
import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.llms import VertexAI

warnings.filterwarnings("ignore")

### 載入模型

你載入的名為 `text-bison@001` 的預訓練文字生成模型。


In [ ]:
vertex_llm_text = VertexAI(model_name="text-bison@001")

## 大文件摘要


### 準備資料檔案

為開始，你將需要下載幾個必備於以下摘要作業的檔案。


In [ ]:
data_folder = p.cwd() / "data"
p(data_folder).mkdir(parents=True, exist_ok=True)

pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = str(p(data_folder, pdf_url.split("/")[-1]))

urllib.request.urlretrieve(pdf_url, pdf_file)

### 從 PDF 中擷取文字

使用 `PdfReader` 從掃描文件擷取文字。


In [ ]:
pdf_loader = PyPDFLoader(pdf_file)
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

## 方法 1：填入

填入是最簡單的方法可將資料傳遞給語言模型。它將文字「塞入」提示中，當作脈絡，如此一來，模型就能夠處理所有相關資訊以取得你要的內容。

在 LangChain 中，你可以將 `StuffDocumentsChain` 作為 `load_summarize_chain` 方法的一部分使用。你需要做的就是設定鏈的 `chain_type` 為 `stuff`。


### 指令設計有`填充`鏈


In [ ]:
prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

### 重試
使用 `stuff` 方法啟動鏈並處理三頁的文件。


In [ ]:
stuff_chain = load_summarize_chain(vertex_llm_text, chain_type="stuff", prompt=prompt)

In [ ]:
three_pages = pages[:3]

In [ ]:
three_pages

In [ ]:
try:
    print(stuff_chain.run(three_pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

正如你可以看到的，使用` stuff `方法，你可以透過一次 API 呼叫傳遞所有數據來總結整個文件內容。

取決於 LLM 的語境長度，`stuff` 方法將無法正常工作，因為它會導致提示大於語境長度。


In [ ]:
try:
    print(stuff_chain.run(pages))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

如預期，程式碼回傳預期訊息。


### 考量事項

「填充」方法是一種摘要文件的方法，在單一呼叫中將整個文件傳輸到大型語言模型 (LLM)。此方法有優點也有缺點。

填充方法僅需要呼叫 LLM 一次，這比需要多次呼叫的其他方法要快。在摘要文本時，LLM 可以一次存取所有資料，這可能會產生更好的摘要。

但是，LLM 有上下文長度，這是單次呼叫中可處理的最大代幣數。如果文件大於上下文長度，則填充方法將不起作用。而且，填充方法不適合摘要大型文件，因為它很慢，而且可能無法產生好的摘要。

讓我們探討其他方法，以協助處理比 LLM 的背景長度限制還要長的文字。


## 方法 2: MapReduce

`MapReduce` 方法實作多階段摘要。它是透過先摘要較小的文字塊，然後將這些摘要組合成單一摘要摘要大型文字區塊的一種技術。

在 LangChain 中，你可以在 `load_summarize_chain` 方法中使用 `MapReduceDocumentsChain`。你需要做的是將鏈的 `chain_type` 設定為 `map_reduce`。


### 使用 `MapReduce` 鏈設計提示

在我們的範例中，有一個 32 頁的文件，你需要摘要。

使用 LangChain，`map_reduce` 鏈最多將文件分成 1024 個トークン塊。接著，在每個塊上執行你定義的初始提示來產生該塊的摘要。在下面的範例中，使用了以下第一階段或對應提示。

```撰寫以下三引號分隔的文字的簡潔摘要。以涵蓋文字要點的項目符號回覆。
'''{text}'''。項目符號摘要：```

一旦所有塊的摘要產生後，就會執行不同的提示，將這些摘要合併成單一摘要。在下面的範例中，使用了以下第二階段或合併提示。

```撰寫包含所有個別摘要要點的整個文件摘要。```


In [ ]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

### 使用 MapReduce 方法產生摘要

在定義提示之後，請初始化關聯的 `map_reduce_chain`。


In [ ]:
map_reduce_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

然後，使用鏈條產生摘要。注意 LangChain 預設使用一個 Token 數量限制為 1024 的 Token 分詞器 (來自 transformer 函式庫)。


In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

在摘要產生後，你可以透過整理輸入文件與其相關的輸出，在 Pandas Dataframe 中驗證它們。


In [ ]:
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_mp_data.append(output)

In [ ]:
pdf_mp_summary = pd.DataFrame.from_dict(final_mp_data)
pdf_mp_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the dataframe by filename and page_number
pdf_mp_summary.reset_index(inplace=True, drop=True)
pdf_mp_summary.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_mp_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_mp_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_mp_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_mp_summary["file_name"].iloc[index])

### 考量

有了 `MapReduce` 方法，該模型能以平行處理克服 `Stuffing` 方法的脈絡限制，進而摘要長篇文件。

然而，`MapReduce` 需要多次呼叫模型，並有可能在頁面之間失去脈絡。

要解決這個問題，你可以嘗試另一種方法來一次摘要多個頁面。


## 方法 3：Refine

Refine 方法是處理大型文件摘要的另一種方法。其運作方式是先在少量資料上執行初始提示，生成一些輸出。然後，針對後續的每一份文件，會輸入前一文件的輸出和新文件，並要求 LLM 根據新文件修正輸出。

在 LangChain 中，你可以在 load_summarize_chain 方法中使用 `MapReduceDocumentsChain`。你需要做的就是將鏈的 `refine` 設定為 `chain_type`。


### 以 `Refine` 鏈設計提示

使用 LangChain，`refine` 鏈需要兩個提示。

用於產生後續任務輸出之問題提示。用於根據產生之內容修正輸出的修正提示。

在此範例中，問題提示為：

```
請提供以下文字之摘要。
TEXT: {text}
SUMMARY:
```

而修正提示為：

```
用三個反引號分隔，撰寫以下文字的簡潔摘要。
以項目符號回覆，涵蓋文字中的重點。
```{text}```
項目符號摘要：
```


In [ ]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

### 使用 Refine 方法生成摘要

在你定義提示後，可以使用「refine」鏈類型啟動摘要鏈。


In [ ]:
refine_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

然後，你使用總結鏈，使用「精煉」方法來總結文件。


In [ ]:
refine_outputs = refine_chain({"input_documents": pages})

以下你可以看到總結的結果。


In [ ]:
final_refine_data = []
for doc, out in zip(
    refine_outputs["input_documents"], refine_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["concise_summary"] = out
    final_refine_data.append(output)

In [ ]:
pdf_refine_summary = pd.DataFrame.from_dict(final_refine_data)
pdf_refine_summary = pdf_mp_summary.sort_values(
    by=["file_name", "page_number"]
)  # sorting the datafram by filename and page_number
pdf_refine_summary.reset_index(inplace=True, drop=True)
pdf_refine_summary.head()

In [ ]:
index = 3
print("[Context]")
print(pdf_refine_summary["chunks"].iloc[index])
print("\n\n [Simple Summary]")
print(pdf_refine_summary["concise_summary"].iloc[index])
print("\n\n [Page number]")
print(pdf_refine_summary["page_number"].iloc[index])
print("\n\n [Source: file_name]")
print(pdf_refine_summary["file_name"].iloc[index])

### 考慮


簡單來說，採用 LLM 的文字摘要 Refine 方法可以匯入更多相關脈絡，且資料損失可能比 Map Reduce 少。不過，與 Stuffing 相比，Refine 需要更多次呼叫 LLM，而且這些呼叫不是獨立的，表示它們無法平行處理。此外，對於文件的排序順序也有一定的依賴性。最新的文件可能變得更相關，因為此方法容易受 recency bias 影響。


## 結論


在本筆記本中，你將了解使用 LangChain 和 PaLM API 總結長篇文件的方法。你在本筆記本中看到的方法是你能使用的可能性之一。例如，還有另一種稱爲Map-Rerank 方法，它包括在每個數據塊上運行一個初始提示，不僅嘗試完成一個任務，而且還給出其答案如何確定的評分。然後根據此評分對響應進行排序，並返回最高評分。

話雖如此，重要的是強調，根據你的需要，你可以考慮使用純粹基礎模型，並使用自訂框架來建立生成式 ai 應用程式。

以下是使用帶有自訂框架的基礎模型的一些好處：

 - 更靈活地使用不同的 LLM、提示範本、文件處理策略等來實現應用程式。

 - 更能根據你的情境自訂生成式應用程式。

 - 更佳的效能以改善應用程式的延遲和可擴充性。
